In [2]:
import pandas as pd
import numpy as np
import time
import datetime
import chromedriver_autoinstaller
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito") # 시크릿 모드
chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
try: # 크롬 드라이버
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
# WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
# driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')


[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 109MB/s]


In [ ]:
country_name = input()
url = f"https://{country_name}.bizin.eu/"
driver.get(url)

In [4]:
areas_lst = {}
areas_href = {}
areas = driver.find_element(By.XPATH, '//*[@id="main_categories_dk"]')
for area in areas.find_elements(By.TAG_NAME, "div"):
    try:
        name = area.text
        pattern = r'\d+'
        numbers = re.findall(pattern, name)
        numbers = [int(n) for n in numbers][0]
        replace_pattern = r'\s*\(\d+\)'
        replace_name = re.sub(replace_pattern, '', name)
        cnt = int(numbers) 
        areas_lst[replace_name] = cnt
        href = area.find_element(By.TAG_NAME, "a").get_attribute("href")
        areas_href[replace_name] = href
    except:
        print(f"fail {area.text}")
        pass

In [76]:
max_area = max(areas_lst, key=areas_lst.get)
print(areas_href[max_area])

https://de.bizin.eu/eng/cat/activities-of-membership-organizations-nec--excluding-social-services--culture-and-sports--1958


In [157]:
first = 0
for num in range(1, areas_lst[max_area]//20):
    driver.get(f"{areas_href[max_area]}?p={num}")
    try: org_lst = driver.find_element(By.XPATH, "/html/body/div[3]/main/div/div[6]/div[1]")
    except : org_lst = driver.find_element(By.XPATH, "/html/body/div[2]/main/div/div[6]/div[1]")
    href_lst = []
    for href in org_lst.find_elements(By.TAG_NAME, "div"):
        try:
            href_lst.append(href.find_element(By.TAG_NAME, "a").get_attribute("href"))
        except:
            pass
    href_lst = [i for i in href_lst if "bizin" in i]
    href_lst = list(set(href_lst))
    for idx in tqdm(href_lst):
        try:
            driver.get(idx)
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            if first == 0:
                df = pd.read_html(str(soup.find('table')))[0]
                columns = df.transpose().iloc[0]
                df = df.T
                df.columns = columns
                df = df.drop(0, axis=0)
                first += 1
            else:
                append_df = pd.read_html(str(soup.find('table')))[0]
                append_df_columns = append_df.transpose().iloc[0]
                append_df = append_df.T
                append_df.columns = list(append_df_columns)
                append_df = append_df.drop(0, axis=0)
                df = df.append(append_df)
        except NoSuchElementException as C:
            print(C)


  5%|▌         | 1/20 [00:02<00:41,  2.17s/it]C:\Users\user\AppData\Local\Temp\ipykernel_59888\97856284.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(append_df)
 10%|█         | 2/20 [00:04<00:38,  2.12s/it]C:\Users\user\AppData\Local\Temp\ipykernel_59888\97856284.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(append_df)
 15%|█▌        | 3/20 [00:06<00:37,  2.23s/it]C:\Users\user\AppData\Local\Temp\ipykernel_59888\97856284.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(append_df)
 20%|██        | 4/20 [00:08<00:35,  2.25s/it]C:\Users\user\AppData\Local\Temp\ipykernel_59888\97856284.py:32: FutureWarning: The frame.append method is deprecated and will be removed 

ProtocolError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))